# Fun Visualisations of the Tracked Events using `plotX`
cf. Plots when exchange for Basic Tracking (i.e. `extreme_events_basic_gridded.zarr`)

In [ ]:
import xarray as xr
import numpy as np
import dask
import matplotlib.pyplot as plt
from getpass import getuser
from pathlib import Path

import marEx
import marEx.helper as hpc

In [ ]:
# Start Dask Cluster
client = hpc.start_local_cluster(n_workers=16, n_threads=1)

In [ ]:
# Directories
scratch_dir = Path('/scratch') / getuser()[0] / getuser()

file_name = scratch_dir / 'mhws' / 'extreme_events_merged_gridded.zarr'
plot_dir  = scratch_dir / 'mhws' / 'plots'

In [ ]:
# Import Tracked Extremes DataSet
extreme_events_ds = xr.open_zarr(str(file_name), chunks={})
extreme_events_ds

<xarray.Dataset> Size: 69GB
Dimensions:       (ID: 11605, time: 13879, lat: 721, lon: 1440, component: 2,
                   sibling_ID: 13)
Coordinates:
  * ID            (ID) int32 46kB 1 2 3 4 5 6 ... 11601 11602 11603 11604 11605
  * lat           (lat) float64 6kB -90.0 -89.75 -89.5 ... 89.5 89.75 90.0
  * lon           (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time          (time) datetime64[ns] 111kB 2002-01-01T23:59:00 ... 2039-12...
Dimensions without coordinates: component, sibling_ID
Data variables:
    ID_field      (time, lat, lon) int32 58GB dask.array<chunksize=(25, 721, 1440), meta=np.ndarray>
    area          (time, ID) float32 644MB dask.array<chunksize=(868, 726), meta=np.ndarray>
    centroid      (component, time, ID) float32 1GB dask.array<chunksize=(1, 868, 726), meta=np.ndarray>
    global_ID     (time, ID) int32 644MB dask.array<chunksize=(868, 726), meta=np.ndarray>
    merge_ledger  (time, ID, sibling_ID) int32 8GB dask.array<chunksize=(25, 11605, 13), meta=np.ndarray>
    presence      (time, ID) bool 161MB dask.array<chunksize=(868, 1451), meta=np.ndarray>
    time_end      (ID) datetime64[ns] 93kB dask.array<chunksize=(11605,), meta=np.ndarray>
    time_start    (ID) datetime64[ns] 93kB dask.array<chunksize=(11605,), meta=np.ndarray>
Attributes: (12/13)
    N_blobs_final:           11605
    N_blobs_unfiltered:      311435
    R_fill:                  8
    T_fill:                  2
    accepted_area_fraction:  0.9172714835387928
    allow_merging:           1
    ...                      ...
    area_threshold:          750.0
    multi_parent_merges:     6221
    nn_partitioning:         1
    overlap_threshold:       0.5
    rejected_area_fraction:  0.08272851646120712
    total_merges:            27270

In [ ]:
# Extract the Events ID Field
ID_field = extreme_events_ds.ID_field
ID_field

<xarray.DataArray 'ID_field' (time: 13879, lat: 721, lon: 1440)> Size: 58GB
dask.array<open_dataset-ID_field, shape=(13879, 721, 1440), dtype=int32, chunksize=(25, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon      (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time     (time) datetime64[ns] 111kB 2002-01-01T23:59:00 ... 2039-12-31T2...

## Make a Movie using `plotX.animate`
N.B.: Making `dask`-powered movies using `animate()` cannot be threaded !

In [ ]:
# Choose a subset of the ID field
ID_field_subset = ID_field.sel(time=slice('2024-01-01', '2026-01-01'))

# Setup Plotting Configuration & Animate
config = marEx.PlotConfig(plot_IDs=True)
ID_field_subset.plotX.animate(configplot_dir=plot_dir, file_name='movie_ID_field_gridded')

In [ ]:
%%HTML
<video width="640" height="480" controls>
    <source src="./movie_ID_field_gridded.mp4" type="video/mp4">
    Your browser does not support the video tag.
</video>

## Plot Consecutive Days of Extreme Events

In [ ]:
# Choose a subset of the ID field
ID_field_subset = ID_field.sel(time=slice('2025-01-01', '2025-01-06'))

# Setup Plotting Configuration
config = marEx.PlotConfig(plot_IDs=True)
fig, ax = ID_field_subset.plotX.multi_plot(config, col='time', col_wrap=3);

## Plot Consecutive Months of Extreme Events

In [ ]:
# Choose the first day of each month from a subset of the ID field
ID_field_subset_day1 = ID_field.sel(time=slice('2025-01-01', '2025-12-31')).resample(time='MS').first()

# Setup Plotting Configuration
config = marEx.PlotConfig(plot_IDs=True)
fig, ax = ID_field_subset_day1.plotX.multi_plot(config, col='time', col_wrap=3);


## Plot Global Extreme Event Frequency

In [ ]:
# Calculate the frequency of Event Occurrence
event_frequency = (ID_field > 0).astype(float).mean('time')

# Setup Plotting Configuration
config = marEx.PlotConfig(var_units='MHW Frequency', cmap='hot_r', cperc=[0,96], grid_labels=True)
fig, ax = event_frequency.plotX.single_plot(config)

## Find & Plot a Few of the Longest Events

In [ ]:
events_duration = extreme_events_ds.time_end - extreme_events_ds.time_start
longest_events = events_duration.sortby(events_duration, ascending=False).ID

for ID in longest_events[:10].values:
    print(f"ID: {ID:<6}   Start Day: {extreme_events_ds.time_start.sel(ID=ID).dt.strftime('%Y-%m-%d').values}  -->  Duration: {events_duration.sel(ID=ID).dt.days.values:<4} days")

ID: 8331     Start Day: 2028-10-22  -->  Duration: 520  days
ID: 8332     Start Day: 2028-10-22  -->  Duration: 454  days
ID: 8197     Start Day: 2028-05-25  -->  Duration: 444  days
ID: 8325     Start Day: 2028-10-15  -->  Duration: 442  days
ID: 8622     Start Day: 2029-10-22  -->  Duration: 411  days
ID: 1912     Start Day: 2008-05-12  -->  Duration: 408  days
ID: 8439     Start Day: 2029-03-05  -->  Duration: 337  days
ID: 371      Start Day: 2003-03-27  -->  Duration: 328  days
ID: 6858     Start Day: 2024-04-28  -->  Duration: 294  days
ID: 4246     Start Day: 2016-01-09  -->  Duration: 285  days


In [ ]:
# Calculate the local duration of the longest events
long_events = extreme_events_ds == (longest_events[:9]).chunk({'ID':1})
long_events_local_duration = (long_events > 0).sum('time')

# Setup Plotting Configuration
config = marEx.PlotConfig(var_units='Duration (days)', cmap='hot_r', cperc=[0, 100])
fig, ax = long_events_local_duration.plotX.multi_plot(config, col='ID', col_wrap=3);

## ID Maximum and Mean Event Area
Units for `extreme_events_ds.area` when using gridded data is 'cells'. Here we must convert to km².

In [ ]:
# Calculate grid areas
R_earth = 6371. #km
resolution = 0.25 #deg

lat_r, lon_r, dlat, dlon = np.radians(extreme_events_ds.lat), np.radians(extreme_events_ds.lon), np.radians(resolution), np.radians(resolution)
grid_area = (R_earth**2 * np.abs(np.sin(lat_r + dlat/2) - np.sin(lat_r - dlat/2)) * dlon).astype(np.float32)

In [ ]:
# This isn't particularly efficient, but gridded tracking has so many lat/lon grid-related biases anyways...
IDs = xr.DataArray(extreme_events_ds.ID).chunk(chunks={'ID':200})
areas = ((extreme_events_ds == IDs).sum('lon') * grid_area).sum('lat')

areas_max = areas.max().compute()
print(f'Maximum object area: {areas_max.values:.0f} km²')

areas_mean = areas.mean().compute()
print(f'Mean object area: {areas_mean.values:.0f} km²')

## Time-Series Plot of Events Area

In [ ]:
area_mean = areas.mean('ID').resample(time='ME').mean()

area_10 = areas.reduce(np.nanpercentile, q=10, dim='ID').resample(time='ME').mean()
area_90 = areas.reduce(np.nanpercentile, q=90, dim='ID').resample(time='ME').mean()

plt.figure(figsize=(15, 6))
area_mean.plot(label='Mean Area', color='k', lw=2)
plt.fill_between(area_mean.time.values, area_10, area_90, alpha=0.5)
plt.ylim([0, 8e7])
plt.ylabel(r'Event Area [km$^2$]');

## Hovmüller Plot of MHW Monthly Frequency vs Latitude

In [ ]:
spatial_presence = (ID_field > 0).mean(dim='lon').resample(time='ME').mean()

fig, ax = plt.subplots(figsize=(15, 6))
im = spatial_presence.plot(ax=ax, cmap='hot', x='time',
    cbar_kwargs={
        'label': 'MHW Presence Frequency',
        'extend': 'both'})

ax.set_xlabel('Time')
ax.set_ylabel('Latitude')
ax.grid(True, linestyle='--', alpha=0.6)


## Seasonal Cycle of MHW vs Latitude

In [ ]:
spatial_presence_seasonal = (ID_field > 0).mean(dim='lon').groupby('time.dayofyear').mean()

fig, ax = plt.subplots(figsize=(15, 6))
im = spatial_presence_seasonal.plot(ax=ax, cmap='hot', x='dayofyear',
    cbar_kwargs={
        'label': 'MHW Seasonal Frequency',
        'extend': 'both'})

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
days_per_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
month_positions = np.cumsum([0] + days_per_month[:-1]) + 15

ax.set_xticks(month_positions)
ax.set_xticklabels(months)
ax.set_xlabel('Month')
ax.set_ylabel('Latitude')
ax.grid(True, linestyle='--', alpha=0.6)
